# Imports

In [280]:
import pandas as pd
import numpy as np
import svgwrite

# Data

In [524]:
vac_df = pd.read_csv('/Users/santa/Projects/vaccine-world-cup/data/data.csv').drop(0)
vac_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10563 entries, 1 to 10563
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   location                             10563 non-null  object 
 1   iso_code                             10563 non-null  object 
 2   date                                 10563 non-null  object 
 3   total_vaccinations                   6621 non-null   float64
 4   people_vaccinated                    6006 non-null   float64
 5   people_fully_vaccinated              4290 non-null   float64
 6   daily_vaccinations_raw               5631 non-null   float64
 7   daily_vaccinations                   10373 non-null  float64
 8   total_vaccinations_per_hundred       6621 non-null   float64
 9   people_vaccinated_per_hundred        6006 non-null   float64
 10  people_fully_vaccinated_per_hundred  4290 non-null   float64
 11  daily_vaccinations_per_milli

In [525]:
vac_df.total_vaccinations = pd.to_numeric(vac_df.total_vaccinations)
vac_df.date = pd.to_datetime(vac_df.date)

In [526]:
vac_df[vac_df.location=='World'].sample(1)

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
10447,World,OWID_WRL,2021-01-23,63993117.0,40346216.0,5446950.0,2991494.0,3485862.0,0.82,0.52,0.07,447.0


In [527]:
pop_df = pd.read_csv('/Users/santa/Projects/vaccine-world-cup/data/pop_data.csv')
pop_df.head()

,entity,iso_code,year,population
0,Afghanistan,AFG,2020,38928341
1,Albania,ALB,2020,2877800
2,Algeria,DZA,2020,43851043
3,American Samoa,ASM,2020,55197
4,Andorra,AND,2020,77265


In [528]:
df = vac_df[['location','date','iso_code','people_vaccinated','people_vaccinated_per_hundred']].merge(pop_df[['iso_code','population']], on=['iso_code'], how='inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10045 entries, 0 to 10044
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   location                       10045 non-null  object        
 1   date                           10045 non-null  datetime64[ns]
 2   iso_code                       10045 non-null  object        
 3   people_vaccinated              5657 non-null   float64       
 4   people_vaccinated_per_hundred  5657 non-null   float64       
 5   population                     10045 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 549.3+ KB


# SVG

In [529]:
HEIGHT = 800
WIDTH = 200

In [556]:
df[df.location=='India'].people_vaccinated_per_hundred.max()

4.98

In [573]:
dont_draw_list = ['China']
draw_text_finish_line_list = ["World"]

def get_graph_for_location( location, color='black', hor_scale=1.2, ver_scale=1, emoji="🏳"): 
    fill = color

    format_df = df[df.location=='World'].sort_values('date').reset_index()['date'].to_frame()
    location_df = df[df.location==location].set_index('date').people_vaccinated.fillna(method='bfill').to_frame().reset_index()
    m_df = format_df.merge(location_df, on=['date'], how='left').fillna(0)

    vacc_list = [ HEIGHT-(i * ver_scale) for i in ((m_df.people_vaccinated/10000000).values.tolist())]
    ran_list = [ (i * hor_scale) for i in  list(range(0, len(vacc_list)))]
    path_string = ""
    for i,j in zip(ran_list, vacc_list) :
        path_string += "L "
        path_string += (str(i)+" ")
        path_string += (str(j)+" ")
    print(path_string[:20],":::", path_string[-20:])
    d_string_area = 'M 0 '+str(HEIGHT)+ path_string + 'L '+str(ran_list[-1])+' '+str(HEIGHT)+' z'
    d_string_line = 'M 0 '+str(HEIGHT)+ path_string + ' '
    
    area_svg = "<path fill=\"{1}\" stroke-width=0.5 opacity=0.1 d=\"{0}\"/>".format(d_string_area, color)
    line_svg = "<path fill=\"none\" stroke-width=0.5 stroke=\"{1}\" d=\"{0}\"/>".format(d_string_line, color)

    text_svg = '''
    <text 
        x="{0}" y="{1}" 
        font-family="Verdana" 
        text-anchor="start" 
        font-size="4" 
        transform="scale (0.5,1)" 
        fill="{3}">
        {4}&nbsp;{2}&nbsp;
            <tspan
                font-family="Verdana" 
                text-anchor="start" 
                font-size="3" 
                transform="scale (0.5,1)" 
                fill="#eee">
                {5:.1f}%
            </tspan>
    </text>'''.format(
        ran_list[-1]*2 + 2,
        vacc_list[-1],
        location,
        color,
        emoji,
        df[df.location==location].people_vaccinated_per_hundred.max()
    )

    finish_line_loc = HEIGHT - df[df.location==location].population.values[0]/10000000
    finish_line_svg = '''
    <path 
        fill="none" stroke="{color}" 
        stroke-width="0.5" 
        stroke-dasharray="1,0.5" 
        d="M0 {value} l200 0">
    </path>'''.format( 
        value =  finish_line_loc,
        color = color)

    if location in draw_text_finish_line_list:
        finish_line_svg +='''
        <text x="375" y="{value}" 
        font-family="Verdana" 
        text-anchor="end" font-size="5" transform="scale (0.5,1)" fill="white">World Finish Line</text>'''.format(value=finish_line_loc-1)

    print(location, dont_draw_list)
    if location in dont_draw_list:
        ret_string = "\t {}".format(finish_line_svg)
    else :
        ret_string =  "\t {0}\n\t {1}\n\t {2}\n\t {3}".format(area_svg, line_svg,text_svg, finish_line_svg)

    return ret_string

def group_location (locations = [], colors = []):
    ret_string = ""

    for l, c in zip(locations, colors):
        ret_string += (get_graph_for_location(location=l, color=c, emoji=""))

    return ret_string

def grid_pattern():

    ret_string = '''
        <defs>
            <pattern 
                id="pattern1" 
                patternUnits="userSpaceOnUse" 
                x="0" y="0" width="10" height="10" 
                viewBox="0 0 4 4">
                <path 
                    d="M 0 0 L 0 4 L 4 4 L 4 0 Z" 
                    fill="none" 
                    stroke-width="0.05" 
                    stroke="#575757"></path>
            </pattern>
  
        </defs>
        <g>
            <rect x="0" y="0" 
                  width="100%" 
                  height="100%" 
                  fill="url(#pattern1)" />
        </g>
    '''

    grid_tick_y = ""
    for i in range(5, 80,5):
        grid_tick_y += ''' <text x="5" y="{y}" >{pop:.1f}B</text>'''.format(pop=i/10, y= 800-(i*10))
    grid_tick_y = '''
    <g fill="#aaa" font-family="Verdana" text-anchor="start" font-size="3" transform="scale (0.5,1)" >
        {}
    </g>
    '''.format(grid_tick_y)

    return ret_string + grid_tick_y

def background():
    return '''
        <defs xmlns="http://www.w3.org/2000/svg">
					<linearGradient xmlns="http://www.w3.org/2000/svg" id="gradient-fill" x1="0" y1="0" x2="800" y2="0" gradientUnits="userSpaceOnUse">				
							<stop offset="0" stop-color="#060d20"/>
                            <stop offset="0.14285714285714285" stop-color="#091228"/>
                            <stop offset="0.2857142857142857" stop-color="#091630"/>
                            <stop offset="0.42857142857142855" stop-color="#091a38"/>
                            <stop offset="0.5714285714285714" stop-color="#0a1d40"/>
                            <stop offset="0.7142857142857142" stop-color="#0b2149"/>
                            <stop offset="0.8571428571428571" stop-color="#0c2451"/>
                            <stop offset="1" stop-color="#0e285a"/>
					</linearGradient>
				</defs>

    <rect fill="url(#gradient-fill)" width="200" height="800"/>
    '''

html_string = '''
<html style="width:100%;height:100%;">
<body style="width:100%;height:100%;margin:0;">
<div style="height: 400%; width: 100%; border:2px solid #000;  position: absolute;" >
  <svg height="100%" width="100%" viewBox="0 0 200 800" position="absolute" preserveAspectRatio="none" >
    
    {background}
    {grid_pattern}
    {group_location}
 </svg>
</div>

<div style="height: 70%;width: 35%;top:15%;left:3%;padding-top:2%;background-color: #222;position:relative;
  text-align: center;opacity: 0.5;  border: 5px solid black;">
<h1 style="color:white;text-shadow: 3px 3px black;font-size: 50px;  font-family: "Lucida Console", Monaco, monospace;">  Vaccine World Cup</h1>
<hr>
</div>

</body>
</html>
'''.format(

    group_location = group_location(
    locations=['World','India', 'United States','European Union', 'Africa','South America', 'China'],
    colors=['#cccccc','#FF9933', '#2D89FF' ,'#0055FF','#00c918','#f8ff33',"#DE2910"],
    ),

    grid_pattern = grid_pattern(),

    background = background()
    
    )


f = open('/Users/santa/Projects/vaccine-world-cup/index.html','w')
f.write(html_string)
f.close()

L 0.0 799.9913535 L  ::: L 134.4 762.9674827 
World ['China']
L 0.0 800.0 L 1.2 80 ::: L 134.4 793.1321162 
India ['China']
L 0.0 800.0 L 1.2 80 ::: L 134.4 789.3785076 
United States ['China']
L 0.0 800.0 L 1.2 80 :::  L 134.4 794.377651 
European Union ['China']
L 0.0 800.0 L 1.2 80 ::: L 134.4 799.2413722 
Africa ['China']
L 0.0 800.0 L 1.2 80 ::: L 134.4 796.9131688 
South America ['China']
L 0.0 800.0 L 1.2 80 ::: 800.0 L 134.4 800.0 
China ['China']


In [558]:
df[df.location=="World"].people_vaccinated.max()

370325173.0

In [565]:
grid_tick_y

' <text x="15" y="750" >500M</text> <text x="15" y="700" >1000M</text> <text x="15" y="650" >1500M</text> <text x="15" y="600" >2000M</text> <text x="15" y="550" >2500M</text> <text x="15" y="500" >3000M</text> <text x="15" y="450" >3500M</text> <text x="15" y="400" >4000M</text> <text x="15" y="350" >4500M</text> <text x="15" y="300" >5000M</text> <text x="15" y="250" >5500M</text> <text x="15" y="200" >6000M</text> <text x="15" y="150" >6500M</text> <text x="15" y="100" >7000M</text> <text x="15" y="50" >7500M</text>'